# Método Monte Carlo (Parte II)

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import math
import scipy.stats as stats
import random

## Generamos el Dataframe de Entradas

In [22]:
activities = {
    "N": [1, 2, 3, 4, 5, 6, 7, 8],
    "Actividades": ["Actividad 1", "Actividad 2", "Actividad 3", "Actividad 4", "Actividad 5", "Actividad 6", "Actividad 7", "Actividad n"],
    "Min": [36, 27, 72, 45, 90, 63, 54, 18],
    "+Probable": [40, 30, 80, 50, 100, 70, 60, 20],
    "Max": [50, 37.5, 100, 62.5, 125, 87.5, 75, 25],
    "Ruta_Critica": [1, 0, 1, 0, 1, 0, 0, 1],
}

activities_df = pd.DataFrame(activities)
activities_df = activities_df[activities_df["Ruta_Critica"] == 1]

print(activities_df)

   N  Actividades  Min  +Probable    Max  Ruta_Critica
0  1  Actividad 1   36         40   50.0             1
2  3  Actividad 3   72         80  100.0             1
4  5  Actividad 5   90        100  125.0             1
7  8  Actividad n   18         20   25.0             1


In [23]:
activities_df['Media_(µ)']  = (activities_df['Max'] + 4*activities_df['+Probable'] + activities_df['Min']) / 6
activities_df['σ']          = (activities_df['Max'] - activities_df['Min']) / 6
activities_df['σ^2']        = activities_df['σ']**2
activities_df['α']          = ((activities_df['Media_(µ)'] - activities_df['Min']) / (activities_df['Max'] - activities_df['Min'])) * (((activities_df['Media_(µ)'] - activities_df['Min']) * (activities_df['Max'] - activities_df['Media_(µ)']) / activities_df['σ']**2) - 1)
activities_df['β']          = ((activities_df['Max'] - activities_df['Media_(µ)']) / (activities_df['Media_(µ)'] - activities_df['Min'])) * activities_df['α']

In [24]:
print(activities_df[activities_df["Ruta_Critica"] == 1])

   N  Actividades  Min  +Probable    Max  Ruta_Critica  Media_(µ)         σ  \
0  1  Actividad 1   36         40   50.0             1       41.0  2.333333   
2  3  Actividad 3   72         80  100.0             1       82.0  4.666667   
4  5  Actividad 5   90        100  125.0             1      102.5  5.833333   
7  8  Actividad n   18         20   25.0             1       20.5  1.166667   

         σ^2         α         β  
0   5.444444  2.594752  4.670554  
2  21.777778  2.594752  4.670554  
4  34.027778  2.594752  4.670554  
7   1.361111  2.594752  4.670554  


Calculamos los valores de la Beta-Pert

In [41]:
def gen_dist():
    probabilidades = [random.random() for _ in range(len(activities_df))]

    Beta_Pert = stats.beta.ppf(
        probabilidades,
        activities_df['α'],
        activities_df['β'],
        activities_df['Min'],
        activities_df['Max'],
    )

    return sum(Beta_Pert)

In [42]:
simulaciones = [gen_dist() for i in range(400)]

print(simulaciones)

[314.68666624477464, 276.1799501497217, 285.3918711093046, 404.34929498491704, 346.0989267823978, 302.26074449178105, 321.8492970771156, 282.4680268638919, 364.43587428398257, 319.67040714573795, 355.1882218117009, 275.84912823007505, 345.67438133512655, 367.32380149417656, 312.7204037732857, 282.0567022144146, 302.8544837949386, 264.2027613875373, 329.3481440296435, 360.1056947289345, 340.98706996854867, 356.9520954145079, 348.625808061566, 303.40809799249865, 341.0336895751802, 333.3581320933397, 331.3683154562044, 293.56763325268884, 349.3895735082123, 279.1759622808489, 278.75380922249235, 312.3269066870683, 306.4979420200504, 269.26820442609585, 367.82242447387944, 323.43503798363514, 331.5326430476797, 350.72536210905434, 305.65529295132507, 334.7723986156445, 348.03434628909395, 329.34642317780805, 369.05552554892506, 339.0515269784115, 283.3812936491729, 377.82487120752967, 297.0915709836444, 302.7535109130204, 285.3935027017591, 302.0608092151099, 308.0525695296436, 404.362632